<h2> <center> <font color = 'Magenta'> Random Forest </h2>

In [ ]:
import os
import pandas as pd
import numpy as np

from sklearn.model_selection import RandomizedSearchCV, train_test_split
from sklearn.ensemble import RandomForestClassifier
from scipy.stats import randint

from sklearn import metrics
from itertools import chain
from matplotlib import pyplot as plt

In [ ]:
# Tree Visualisation
from sklearn.tree import export_graphviz
from IPython.display import Image
import graphviz

In [ ]:
# Set working Directory
path = "C:/Users/Windows/OneDrive - Lamar University/Documents/Springfield/Python/"
train=pd.read_csv('C:/Users/Windows/OneDrive - Lamar University/Documents/Springfield/Python/SVR/Train_COP30_v4.csv')# 
test=pd.read_csv('C:/Users/Windows/OneDrive - Lamar University/Documents/Springfield/Python/SVR/Test_COP30_v4.csv')# 
os.chdir(path) 


In [ ]:
train.head(5)

In [ ]:
test.head(5)

In [ ]:
features = ['Lookup_Re_ASPECT', 'Lookup_Re_CI', 'Lookup_Re_MBI','Lookup_Re_MRRTF','Lookup_Re_MRVBF','Lookup_Re_MSP', 'Lookup_Re_NH', 'Lookup_Re_NLCD_v3', 'Lookup_Re_SH', 'Lookup_Re_SLOPE', 'Lookup_Re_TPI', 'Lookup_Re_TWI'] # List of Input features for model
train_X = train[features] # Dataframe of the data features
train_Y = train['value'] # Add Y variable to the input deature datframe
#train_X=train.drop(['value'], axis=1, inplace=True)
#test_X.head(5)
train_X.corr()

In [ ]:
features = ['Lookup_Re_ASPECT', 'Lookup_Re_CI', 'Lookup_Re_MBI','Lookup_Re_MRRTF','Lookup_Re_MRVBF','Lookup_Re_MSP', 'Lookup_Re_NH', 'Lookup_Re_NLCD_v3', 'Lookup_Re_SH', 'Lookup_Re_SLOPE', 'Lookup_Re_TPI', 'Lookup_Re_TWI'] # List of Input features for model
test_X = test[features] # Dataframe of the data features
test_Y = test['value'] # Add Y variable to the input deature datframe
#train_X=train.drop(['value'], axis=1, inplace=True)
#test_X.head(5)
test_X.corr()

In [ ]:
train_Y.head(5)


In [ ]:
train_Y.describe()

In [ ]:
train_X = (train_X-train_X.min()) / (train_X.max()-train_X.min()) # Normalize the data
test_X = (test_X-test_X.min()) / (test_X.max()-test_X.min()) # Normalize the data

In [ ]:
train_X

In [ ]:
test_X

<center> <h2> <font color='blue'> Random Forest </h2>

In [ ]:
# instantiate the model 
rf = RandomForestClassifier(n_estimators=892, max_depth=8 random_state=0)

In [ ]:
# fit the model to training data
rf.fit(train_X, train_Y)

In [ ]:
Y_pred = rf.predict(train_X) # Make Predictions
Yprob = rf.predict_proba(train_X) #test output probabilities
zz = pd.DataFrame(Yprob)
zz.head()

In [ ]:
Y_pred1 = rf.predict(test_X) # Make Predictions
Yprob1 = rf.predict_proba(test_X) #test output probabilities
zz1 = pd.DataFrame(Yprob1)
zz1.head()

In [ ]:
# Export the first three decision trees from the forest

for i in range(3):
    tree = rf.estimators_[i]
    dot_data = export_graphviz(tree,
                               feature_names=train_X.columns,  
                               filled=True,  
                               max_depth=2, 
                               impurity=False, 
                               proportion=True)
    graph = graphviz.Source(dot_data)
    display(graph)

In [ ]:
print("Accuracy:",metrics.accuracy_score(train_Y, Y_pred)) #Accuracy of the model on training data
print("Precision:",metrics.precision_score(train_Y, Y_pred)) #Precision of the model on training data
print("Recall:",metrics.recall_score(train_Y, Y_pred)) #Recall of the model on training data

In [ ]:
print("Accuracy:",metrics.accuracy_score(test_Y, Y_pred1)) #Accuracy of the model on testing data
print("Precision:",metrics.precision_score(test_Y, Y_pred1)) #Precision of the model on testing data
print("Recall:",metrics.recall_score(test_Y, Y_pred1)) #Recall of the model on testing data

In [ ]:
from sklearn.metrics import classification_report
print(classification_report(test_Y, Y_pred1))

In [ ]:
cnf_matrix = metrics.confusion_matrix(train_Y, Y_pred) # Calculate confusion matrix for the model
cnf_matrix # y_train is going be rows (obs), y_pred (predicted) are cols
disp = metrics.ConfusionMatrixDisplay(confusion_matrix=cnf_matrix)
disp.plot() #create visualization of confusion matrix
plt.title('Confusion matrix for training data')
plt.show() # Display the visualisation

In [ ]:
cnf_matrix1 = metrics.confusion_matrix(test_Y, Y_pred1) # Calculate confusion matrix for the model
cnf_matrix1 # y_test is going be rows (obs), y_pred1 (predicted) are cols
disp1 = metrics.ConfusionMatrixDisplay(confusion_matrix=cnf_matrix1)
disp1.plot() #create visualization of confusion matrix
plt.title('Confusion matrix for testing data')
plt.show() # Display the visualisation

In [ ]:
Y_pred_proba = rf.predict_proba(train_X)[::,1]
fpr, tpr, _ = metrics.roc_curve(train_Y,  Y_pred_proba)
auc = metrics.roc_auc_score(train_Y, Y_pred_proba)
plt.plot(fpr,tpr,label="data 1, auc="+str(round(auc,4)))
plt.legend(loc=4)
plt.title('ROC curve for training data')
plt.xlabel('1-Specificity')
plt.ylabel('Sensitivity')
plt.grid()
plt.show()

In [ ]:
Y_pred_proba1 = rf.predict_proba(test_X)[::,1]
fpr, tpr, _ = metrics.roc_curve(test_Y,  Y_pred_proba1)
auc = metrics.roc_auc_score(test_Y, Y_pred_proba1)
plt.plot(fpr,tpr,label="data 1, auc="+str(round(auc,4)))
plt.legend(loc=4)
plt.title('ROC curve for testing data')
plt.xlabel('1-Specificity')
plt.ylabel('Sensitivity')
plt.grid()
plt.show()

In [ ]:
# Create a series containing feature importances from the model and feature names from the training data
feature_importances = pd.Series(best_rf.feature_importances_, index=train_X.columns).sort_values(ascending=False)

# Plot a simple bar chart
feature_importances.plot.bar();

In [ ]:
#Read all data
All_data=pd.read_csv('C:/Users/Windows/OneDrive - Lamar University/Documents/Springfield/Python/SVR/Combine_COP30_v4.csv')#
features = ['Lookup_Re_AHS','Lookup_Re_ASPECT', 'Lookup_Re_CI', 'Lookup_Re_MBI','Lookup_Re_MRRTF','Lookup_Re_MRVBF','Lookup_Re_MSP', 'Lookup_Re_NH', 'Lookup_Re_NLCD_v3', 'Lookup_Re_SH', 'Lookup_Re_SLOPE', 'Lookup_Re_TCA', 'Lookup_Re_TPI', 'Lookup_Re_TWI'] # List of Input features for model
All_data = All_data[features] # Dataframe of the data features
All_data = (All_data-All_data.min()) / (All_data.max()-All_data.min()) # Normalize the data
All_data

In [ ]:
#predict the y variable for all x variables

Y_pred_all = rf.predict(All_data) # predict whole dataset
Yprob_all = rf.predict_proba(All_data) #test output probabilities
zz_all = pd.DataFrame(Yprob_all)
zz_all.head(10)

In [ ]:
Y_pred_proba_all = rf.predict_proba(All_data)[:,1]
Y_pred_proba_all

In [ ]:
# Python3 Program to Create list
# with integers within given range
import numpy as np
def createList(r1, r2):
    return np.arange(r1, r2+1, 1)
     
# Driver Code
r1, r2 = 1, 2254695
listNumber=createList(r1, r2)
listNumb=pd.DataFrame(listNumber, columns = ['number'])

In [ ]:
df_prob = pd.DataFrame(Y_pred_proba_all, columns = ['RF_prob_Y'])
df_prob = pd.concat([listNumb, df_prob], axis=1)
df_prob

In [ ]:
df_prob.to_csv('RandomForest_Springfield.csv')